In [237]:
import numpy as np
import math
import itertools

class Poly:        
    def __init__(self, order=None, coeffs=None, notation=2):        
        if coeffs is None:
            self.coeffs = np.zeros(order + 1)
            self.coeffs[order] = 1
        else:
            self.coeffs = coeffs
            
        #self.coeffs = self.coeffs[:self.get_order()+1]
        self.notation = notation
    
    def add(self, poly):   
        if type(poly) is int:
            poly = Poly(poly)
            
        coeffs = poly.coeffs.copy()
        
        if any([i > self.notation - 1 for i in coeffs]):
            raise Exception("Be consistent with notation of poly")
        
        a, b = [], []
        if len(self.coeffs) < len(coeffs):
            a = coeffs
            b = self.coeffs
        else:
            a = self.coeffs
            b = coeffs
            
        c = np.array(a.copy()).astype(int)
        c[:len(b)] += np.array(b).astype(int)
        
        
        return Poly(coeffs=[i%self.notation for i in c])
        
    def multiply_int(self, number):
        return Poly(coeffs=np.hstack((np.zeros(number),self.coeffs)))
      
    def multiply(self, poly):
        if type(poly) is int:
            poly = Poly(poly)
        
        p1 = np.poly1d(list(reversed(self.coeffs)))
        p2 = np.poly1d(list(reversed(poly.coeffs)))
        
        return Poly(coeffs=list(reversed([i%self.notation for i in np.polymul(p1, p2)])))
    
    def get_order(self):
        arr = [i for i,v in enumerate(self.coeffs) if v == 1]
        if len(arr) == 0:
            return 0
        return max(arr)
        
        
    def divide(self, divider):
        if type(divider) is int:
            divider = Poly(divider)
            
        dividend_order = self.get_order()
        divider_order = divider.get_order()
        
        dividend = self.coeffs.copy()
        divider = divider.coeffs.copy()
        divider_len = len(divider)
        
        
        if divider_order > dividend_order:
            return 0, Poly(coeffs=self.coeffs.copy())
        elif dividend_order == divider_order:
            return 0, 0
        else:
            remainder = []
            result = []
            while True:
                diff = dividend_order - divider_order
                
                #print(diff)
                suffix = np.zeros(len(dividend) - divider_len - diff) if len(dividend) - divider_len - diff > 0 else []
                subtract = np.hstack((np.zeros(diff), divider, suffix))
                
                #print(dividend)
                #print(subtract.tolist())
                remainder.append(diff)
                
                rest = abs(dividend - subtract)
                rest = [i%self.notation for i in rest]
                #print(rest)
                
                if any([i != 0 for i in rest]):
                    if max([i for i,v in enumerate(rest) if v != 0]) >= divider_order :
                        dividend = rest.copy()
                        dividend_order = max([i for i,v in enumerate(dividend) if v != 0])

                        continue
                
                break
                
            result_coeffs = [1 if i in remainder else 0 for i in range(max(remainder)+1)]
            
            return Poly(coeffs=result_coeffs), Poly(coeffs=rest) if any([i!=0 for i in rest]) else 0
                
                
        
    def show(self):
        print(" + ".join(["x^" + str(int(i)) for i,v in enumerate(self.coeffs) if v == 1]), 
              str(self.notation) + " notation")
        
def generate_generator(k):
    for n in range(k + 5, k + 15):
        main = Poly(n).add(0)
        for w in range(1, n-k + 2):
            indices = range(0, n-k + 1)
            arrs = list(itertools.combinations(indices, w))
            for arr in arrs:
                if len(arr) == 1 and arr[0] == 0:
                    continue
                generator = np.zeros(n-k+1)
                generator[list(arr)] = 1
                generator = Poly(coeffs=generator)

                gen_order = generator.get_order()    
                
                if gen_order != n-k:
                    continue
                    

                c, r = main.divide(generator) 

                if r != 0 or c == 0:
                    continue

                c_order = c.get_order()
                mat = np.zeros(shape=(n-k,n))
                c = c.add(k).add(k)
                
                for i in range(n-k):
                    mat[i, n-c_order-i-1:n-i] = list(reversed(c.coeffs))

                bad = False
                for i in range(n):
                    item = mat[:, i]
                    for j in range(n):
                        if i == j:
                            continue
                        column = mat[:, j]

                        if np.array_equal(item, column):
                            bad = True
                if bad:
                    continue

                return generator, mat, n

In [252]:
string = input("Word to encode: ")
k = len(string)
code = [int(i) for i in string]

Word to encode: 1011


In [253]:
gen, mat, n = generate_generator(k)
gen.show()

print(mat, n)

x^0 + x^1 + x^5 + x^6 2 notation
[[ 0.  0.  0.  0.  0.  1.  1.  1.  1.  1.]
 [ 0.  0.  0.  0.  1.  1.  1.  1.  1.  0.]
 [ 0.  0.  0.  1.  1.  1.  1.  1.  0.  0.]
 [ 0.  0.  1.  1.  1.  1.  1.  0.  0.  0.]
 [ 0.  1.  1.  1.  1.  1.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.]] 10


In [254]:
code_poly = Poly(coeffs=code)
code_poly.show()

x^0 + x^2 + x^3 2 notation


In [255]:
shifted = code_poly.multiply(n-k)
shifted.show()
shifted.coeffs = list(np.hstack((shifted.coeffs, np.zeros(n - len(shifted.coeffs)))))

x^6 + x^8 + x^9 2 notation


In [256]:
r, c = shifted.divide(gen)
r.show()
c.show()
r.multiply(gen).add(c).show()

x^0 + x^3 2 notation
x^0 + x^1 + x^3 + x^4 + x^5 2 notation
x^6 + x^8 + x^9 2 notation


In [257]:
encoded = shifted.add(c)
encoded.show()
encoded = encoded.add(1)
encoded.show()

x^0 + x^1 + x^3 + x^4 + x^5 + x^6 + x^8 + x^9 2 notation
x^0 + x^3 + x^4 + x^5 + x^6 + x^8 + x^9 2 notation


In [258]:
dd, dr = encoded.divide(gen)
encoded_fixed = None

if dr == 0:
    print("RESULT!!!")
    encoded_fixed = encoded
else:
    indices = range(0, n)
    for w in range(1, n+1):
        if encoded_fixed:
            break
        arrs = list(itertools.combinations(indices, w))
        for err in arrs:
            print(err)
            coeffs = np.zeros(n)
            coeffs[list(err)] = 1
            err_poly = Poly(coeffs=coeffs)
            
            _, mod = err_poly.divide(gen)
            if mod == 0:
                continue
                
            if all([i == 0 for i in mod.add(dr).coeffs]):
                encoded_fixed = encoded.add(err_poly)
                break


print(encoded_fixed.coeffs[n-k:])

(0,)
(1,)
[1, 0, 1, 1]
